In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.pylab as pl
from matplotlib.colors import ListedColormap
np.set_printoptions(precision=3)
%matplotlib inline

# Comparaison en pratique avec les algorithmes classiques


In [ ]:
class Maze():
    def __init__(self, width=30, height=30, gamma=.9):
        """
        Same initialization as seen in class, except that we create a v0 and apply a pi_greedy to it
        """
        self.width = width-1
        self.height = height-1
        maze_size = (height, width)
        start_cell = (0,0)
        target_cell = (height-1, width-1)

        maze = np.random.binomial(1, p=.15, size=maze_size)
        maze[:round(2*height/3),round(width/4)] = 1
        maze[round(height/3):,round(3*width/4)] = 1
        chl = round(2*height/5)
        chu = round(3*height/5)
        cwl = round(2*width/5)
        cwu = round(3*width/5)
        maze[chl:chu,cwl:cwu] = np.random.binomial(1, p=.5, size=(chu-chl, cwu-cwl))
        maze[start_cell] = 0
        maze[target_cell] = 0
        self.maze = maze
        self.actions = [0,1,2,3]
        self.action_coordinates = {0: np.array([1,0]),# down
                            1: np.array([0,1]), # right
                            2: np.array([-1,0]), # up
                            3: np.array([0,-1])} # left
        self.action_arrows = ['↓', '→', '↑', '←']

        ####ADDED####
        self.states = np.argwhere(maze != 1) #get all the states that are not walls
        self.gamma=gamma
        self.v0 = np.random.random((height, width))
        self.pi = self.pi_greedy(self.v0)
        #############
        self.plot(policy=self.pi, path=True)

    def plot(self, policy=None, path=False):
        grid_size = self.maze.shape
        fig, ax = plt.subplots(figsize=(grid_size[1]+1, grid_size[0]+1))
        im = ax.imshow(self.maze, cmap='Greys', interpolation='nearest', extent=[0, grid_size[1], 0, grid_size[0]], alpha=1)
        ax.set_xticks(np.arange(0, grid_size[1]+1, 1), minor=True)
        ax.set_yticks(np.arange(0, grid_size[0]+1, 1), minor=True)
        ax.grid(which="minor", color='black', linestyle='-', linewidth=1)
        ax.set_xticks([])
        ax.set_yticks([])
        if policy is not None:
            # Plot arrows
            for i in range(grid_size[0]):
                for j in range(grid_size[1]):
                    action = policy[i, j]
                    ax.text(j + 0.5, grid_size[0] - i - 0.5, self.action_arrows[action],
                            ha='center', va='center', fontsize=12, fontweight='bold')
            if path is True:
                cmap = pl.cm.Reds
                my_cmap = cmap(np.arange(cmap.N))
                my_cmap[:,-1] = np.linspace(0, 1, cmap.N)
                my_cmap = ListedColormap(my_cmap)

                im = ax.imshow(self.path_array(policy), cmap=my_cmap, interpolation='nearest', extent=[0, grid_size[1], 0, grid_size[0]], alpha=.5)
        plt.show()

    def path_array(self, policy):
        array = np.zeros((self.height+1, self.width+1), dtype=int)
        s = (0, 0)
        array[s] = 1
        while True :
            _, s_ = self.transition(s, policy[s])
            s_ = tuple(s_)
            if array[s_] == 1 : break
            array[s_] = 1
            if s_[0] == self.height and s_[1] == self.width : break
            s = s_
        return array
    
    def transition(self, s,a):
        """
        Transition function of the maze
        
        Parameters
        ----------
        s : tuple
            State
        a : int
            Action

        Returns
        -------
        (r, s_) : tuple
            r : reward
            s_ : next state
        """
        s_ = s+self.action_coordinates[a]
        if not any(np.array_equal(state, s_) for state in self.states):
            return (0, s)
        
        else:
            if np.array_equal(s_, np.array([self.height, self.width])):
                return (1, s_)
            else:
                return (0, s_)
    
    def pi_greedy(self, v):
        """
        Greedy policy for a given value function
        
        Parameters
        ----------
        v : array
            Value function
        
        Returns
        -------
        pi : array
            Greedy policy (array of actions)
        """
        pi = np.zeros((self.height+1, self.width+1), dtype=int)
        for s in self.states:
            pi[tuple(s)] = np.argmax([r + self.gamma*v[tuple(s_)] for r, s_ in [self.transition(s,a) for a in self.actions]])
        return pi

    def Bpi(self, v, pi):
        """
        Bellman PI operator for a given value function and policy

        Parameters
        ----------
        v : array
            Value function
        pi : array
            Policy (array of actions)

        Returns
        -------
        v_ : array
            New value function
        """
        v_ = np.zeros_like(v)
        for s in self.states:
            r, s_ = self.transition(s, pi[tuple(s)])
            v_[tuple(s)] = r+self.gamma*v[tuple(s_)]
        return v_

    def Bstar(self, v, pi):
        """
        Bellman Star operator for a given value function and policy

        Parameters
        ----------
        v : array
            Value function
        pi (UNUSED) : array 
            Policy (array of actions)

        Returns
        -------
        v_ : array
            New value function
        """
        v_ = np.zeros_like(v)
        for s in self.states:
            pi_s = np.argmax([r + self.gamma*v[tuple(s_)] for r, s_ in [self.transition(s,a) for a in self.actions]])
            r, s_ = self.transition(s, pi_s)
            v_[s] = r + self.gamma*v[s_]
        return v_
    
    def AdaVI(self, eta, B,  graph=False, espilon=1e-6, max_iter=1e3):
        """
        Adaptive VI algorithm
        
        Parameters
        ----------
        eta : float
            Learning rate
        B : function
            Bellman operator
        graph : bool, optional
            If True, plot the convergence of the algorithm
        espilon : float, optional
            Convergence threshold
        max_iter : int, optional
            Maximum number of iterations

        Returns
        -------
        norm_list : list
            List of the norm of the difference between the state-value function at each iteration and the optimal state-value function
        """
        v_list = [] #list of v at each iteration
        sum_square = 0 #sum of the square of the difference between Bv and v
        norm_list = [] #list of the norm of the difference between Bv and v
        pi = self.pi
        v = self.v0
        for _ in range(int(max_iter)):
            diff = B(v,pi)-v
            sum_square += np.linalg.norm(diff)**2
            v_ = v + eta*(diff)/np.sqrt(sum_square)
            if np.amax(np.abs(diff)) < espilon:
                print(f'AdaVI converged for eta={eta}')
                break
            elif graph:
                norm_list.append(np.amax(np.abs(diff))) 
                v_list.append(v_)
            v = v_.copy()
        else:
            print(f'AdaVI not converged for eta={eta}')
        
        if graph:
            plt.semilogy(range(len(norm_list)), norm_list, label=r'ADA-VI pour $\eta$='+str(eta))
            return [np.amax(w-v_list[-1]) for w in v_list]
        
    
    def VI(self, B,  graph=False, espilon=1e-6, max_iter=1e6):
        """
        Value Iteration algorithm

        Parameters
        ----------
        B : function
            Bellman operator
        graph : bool, optional
            If True, plot the convergence of the algorithm
        espilon : float, optional
            Convergence threshold
        max_iter : int, optional
            Maximum number of iterations

        Returns
        -------
        norm_list : list
            List of the norm of the difference between the state-value function at each iteration and the optimal state-value function
        """
        v_list = [] #list of v at each iteration
        norm_list = [] #list of the norm of the difference between Bv and v
        pi = self.pi
        v = self.v0
        for _ in range(int(max_iter)):
            v_ = B(v,pi)
            if np.amax(v-v_) < espilon:
                print('VI converged')
                break
            elif graph:
                norm_list.append(np.amax(v-v_))
                v_list.append(v_)
            v = v_.copy()
        else:
            print('VI not converged')
        
        if graph:
            plt.semilogy(range(len(norm_list)), norm_list, label=r'VI')
            return [np.amax(w-v_list[-1]) for w in v_list]
        

In [ ]:
np.random.seed(420)
maze = Maze(10,10)

In [ ]:
v_list = []
etas = [.7, 1, 3, 5, 7, 10, 50, 100]
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
for eta in etas:
    v_list.append(maze.AdaVI(eta, maze.Bpi, graph=True))
v_list.append(maze.VI(maze.Bpi, graph=True))
plt.ylabel(r'$||v_k- B_\pi v_k||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.legend()

plt.subplot(1, 2, 2)
for i, eta in enumerate(etas):
    plt.semilogy(range(len(v_list[i])), v_list[i], label=r'ADA-VI, $\eta=$'+str(eta))
plt.semilogy(range(len(v_list[-1])), v_list[-1], label=r'VI')
plt.legend()
plt.ylabel(r'$||v_k- v_\pi||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.suptitle(r'Comparaison de convergence de Ada-VI$^{(V)}_\pi$ et VI$^{(V)}_\pi$')
plt.tight_layout()
plt.subplots_adjust(wspace=0.4)
plt.show()

In [ ]:
v_list = []
etas = [.7, 1, 3, 5, 7, 10, 50, 100]
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
for eta in etas:
    v_list.append(maze.AdaVI(eta, maze.Bstar, graph=True))
v_list.append(maze.VI(maze.Bstar, graph=True))
plt.ylabel(r'$||v_k- B_* v_k||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.legend()

plt.subplot(1, 2, 2)
for i, eta in enumerate(etas):
    plt.semilogy(range(len(v_list[i])), v_list[i], label=r'ADA-VI, $\eta=$'+str(eta))
plt.semilogy(range(len(v_list[-1])), v_list[-1], label=r'VI')
plt.legend()
plt.ylabel(r'$||v_k- v_*||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.suptitle(r'Comparaison de convergence de Ada-VI$^{(V)}_*$ et VI$^{(V)}_*$')
plt.tight_layout()
plt.subplots_adjust(wspace=0.4)
plt.show()

# Extensions


## Itérations de fonctions action-valeur


In [ ]:
class Maze_Actions(Maze):
    def __init__(self, width=30, height=30, gamma=.9, subsets=5):
        super().__init__(width, height, gamma)
        self.q0 = np.array([np.random.random((height, width)) for _ in self.actions])

    def AdaVI_action(self, eta, B, graph=False, espilon=1e-6, max_iter=1e3):
        """
        Adaptive VI algorithm for action-value functions

        Parameters
        ----------
        eta : float
            Learning rate
        B : function
            Bellman operator
        graph : bool, optional
            If True, plot the convergence of the algorithm
        espilon : float, optional
            Convergence threshold
        max_iter : int, optional
            Maximum number of iterations

        Returns
        -------
        norm_list : list
            List of the norm of the difference between the action-value function at each iteration and the optimal action-value function
        """
        q_list = [] #list of q at each iteration
        sum_square = 0 #sum of the square of the difference between Bq and q
        norm_list = [] #list of the norm of the difference between Bq and q
        pi = self.pi
        q = self.q0
        for _ in range(int(max_iter)):
            diff = B(q,pi)-q
            sum_square += np.linalg.norm(diff)**2
            q_ = q + eta*(diff)/np.sqrt(sum_square)
            if np.amax(np.abs(diff)) < espilon:
                print(f'AdaVI converged for eta={eta}')
                break
            elif graph:
                norm_list.append(np.amax(np.abs(diff)))  
                q_list.append(q_)
            q = q_.copy()
        else:
            print(f'AdaVI not converged for eta={eta}')
        
        if graph:
            plt.semilogy(range(len(norm_list)), norm_list, label=r'ADA-VI pour $\eta=$'+str(eta))
            return [np.amax(p-q_list[-1]) for p in q_list]
            
        return q_list
            
    def VI_action(self, B, graph=False, espilon=1e-6, max_iter=1e3):
        """
        Value Iteration algorithm for action-value functions

        Parameters
        ----------
        B : function
            Bellman operator
        graph : bool, optional
            If True, plot the convergence of the algorithm
        espilon : float, optional
            Convergence threshold
        max_iter : int, optional
            Maximum number of iterations

        Returns
        -------
        norm_list : list
            List of the norm of the difference between the action-value function at each iteration and the optimal action-value function
        """
        q_list = [] #list of q at each iteration
        norm_list = [] #list of the norm of the difference between Bq and q
        pi = self.pi
        q = self.q0
        for _ in range(int(max_iter)):
            q_ = B(q,pi)
            if np.amax(np.abs(q_-q)) < espilon:
                print(f'VI converged')
                break
            elif graph:
                norm_list.append(np.amax(np.abs(q_-q))) 
                q_list.append(q_)
            q = q_.copy()
        else:
            print(f'VI not converged')
        
        if graph:
            plt.semilogy(range(len(norm_list)), norm_list, label=r'VI')
            return [np.amax(p-q_list[-1]) for p in q_list]
    
    def Bpi_action(self, q, pi):
        """
        Bellman PI operator for action-value functions

        Parameters
        ----------
        q : array
            Action-value function
        pi : array
            Policy (array of actions)

        Returns
        -------
        q_ : array
            New action-value function
        """
        q_ = np.zeros_like(q)
        for s in self.states:
            for a in self.actions:
                r, s_ = self.transition(s, a)
                q_[a][tuple(s)] = r + self.gamma*q[pi[tuple(s_)]][tuple(s_)]
        return q_
    
    def Bstar_action(self, q, pi):
        """
        Bellman Star operator for action-value functions

        Parameters
        ----------
        q : array
            Action-value function
        pi (UNUSED) : array
            Policy (array of actions)

        Returns
        -------
        q_ : array
            New action-value function
        """
        q_ = np.zeros_like(q)
        for s in self.states:
            for a in self.actions:
                r, s_ = self.transition(s, a)
                q_[a][tuple(s)] = r+self.gamma*np.max([q[a_][tuple(s_)] for a_ in self.actions])
        return q_

In [ ]:
np.random.seed(420)
maze_actions = Maze_Actions(10,10)

In [ ]:
q_list = []
etas = [3, 5, 7, 10, 50]
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
for eta in etas:
    q_list.append(maze_actions.AdaVI_action(eta, maze_actions.Bpi_action, graph=True))
q_list.append(maze_actions.VI_action(maze_actions.Bpi_action, graph=True))
plt.ylabel(r'$||q_k- B_\pi q_k||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.legend()

plt.subplot(1, 2, 2)
for i, eta in enumerate(etas):
    plt.semilogy(range(len(q_list[i])), q_list[i], label=r'ADA-VI, $\eta=$'+str(eta))
plt.semilogy(range(len(q_list[-1])), q_list[-1], label=r'VI')
plt.legend()
plt.ylabel(r'$||q_k- q_\pi||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.suptitle(r'Comparaison de convergence de Ada-VI$^{(Q)}_\pi$ et VI$^{(Q)}_\pi$')
plt.tight_layout()
plt.subplots_adjust(wspace=0.4)
plt.show()

In [ ]:
q_list = []
etas = [3, 5, 7, 10, 50]
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
for eta in etas:
    q_list.append(maze_actions.AdaVI_action(eta, maze_actions.Bstar_action, graph=True))
q_list.append(maze_actions.VI_action(maze_actions.Bstar_action, graph=True))
plt.ylabel(r'$||q_k- B_* q_k||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.legend()

plt.subplot(1, 2, 2)
for i, eta in enumerate(etas):
    plt.semilogy(range(len(q_list[i])), q_list[i], label=r'ADA-VI, $\eta=$'+str(eta))
plt.plot(range(len(q_list[-1])), q_list[-1], label=r'VI')
plt.legend()
plt.ylabel(r'$||q_k- q_*||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.suptitle(r'Comparaison de convergence de Ada-VI$^{(Q)}_*$ et VI$^{(Q)}_*$')
plt.tight_layout()
plt.subplots_adjust(wspace=0.4)
plt.show()

## Itérations asynchrones


In [ ]:
class Maze_Asynch(Maze):
    def __init__(self, width=30, height=30, gamma=.9, subsets=5):
        super().__init__(width, height, gamma)
        self.subsets=np.array_split(np.argwhere(self.maze != 2), subsets) #on all states because if the wall is not updated, the infinite norm of v-Bv become a constant
    
    def AdaVI_asynch(self, eta, B, graph=False, espilon=1e-6, max_iter=1e3):
        """
        Adaptive VI algorithm with asynchronous updates
        Be careful, one iteration is one update of all components of the state-value function.

        Parameters
        ----------
        eta : float
            Learning rate
        B : function
            Bellman operator
        graph : bool, optional
            If True, plot the convergence of the algorithm
        espilon : float, optional
            Convergence threshold
        max_iter : int, optional
            Maximum number of iterations
        
        Returns
        -------
        norm_list : list
            List of the norm of the difference between the state-value function at each iteration and the optimal state-value function
        """
        v_list = [] #list of v at each iteration
        sum_square = 0 #sum of the square of the difference between Bv and v
        norm_list = []  #list of the norm of the difference between Bv and v
        pi = self.pi
        v = self.v0
        for _ in range(int(max_iter)):
            v_ = v.copy()
            sum_square += np.linalg.norm(B(v_,pi)-v_)**2
            for subset in self.subsets:
                diff = B(v_,pi)-v_
                for s in subset:
                    v_[tuple(s)] = v[tuple(s)] + eta*(diff[tuple(s)])/np.sqrt(sum_square)
            if np.amax(np.abs(diff)) < espilon:
                print(f'AdaVI converged for eta={eta}')
                break
            elif graph:
                norm_list.append(np.amax(np.abs(diff)))
                v_list.append(v_)
            v = v_
        else:
            print(f'AdaVI not converged for eta={eta}')
        
        if graph:
            plt.semilogy(range(len(norm_list)), norm_list, label=r'ADA-VI asynch, $\eta=$'+str(eta))
            return [np.amax(w-v_list[-1]) for w in v_list]
            
    def VI_asynch(self, B, graph=False, espilon=1e-6, max_iter=1e3):
        """
        Value Iteration algorithm with asynchronous updates
        Be careful, one iteration is one update of all components of the state-value function.

        Parameters
        ----------
        B : function
            Bellman operator
        graph : bool, optional
            If True, plot the convergence of the algorithm
        espilon : float, optional
            Convergence threshold
        max_iter : int, optional
            Maximum number of iterations

        Returns
        -------
        norm_list : list
            List of the norm of the difference between the state-value function at each iteration and the optimal state-value function
        """
        v_list = [] #list of v at each iteration
        norm_list = [] #list of the norm of the difference between Bv and v
        pi = self.pi
        v = self.v0
        for _ in range(int(max_iter)):
            v_ = v.copy()
            for subset in self.subsets:
                B_ = B(v_, pi)
                for s in subset:
                    v_[tuple(s)] = B_[tuple(s)]
            if np.amax(np.abs(v_-v)) < espilon:
                print(f'VI converged')
                break
            elif graph:
                norm_list.append(np.amax(np.abs(v_-v))) 
                v_list.append(v_)
            v = v_
        else:
            print(f'VI not converged')
        
        if graph:
            plt.semilogy(range(len(norm_list)), norm_list, label=r'VI asynch')
            return [np.amax(w-v_list[-1]) for w in v_list]

In [ ]:
np.random.seed(420)
maze_asynch = Maze_Asynch(10, 10)

In [ ]:
v_list = []
etas = [3, 5, 7, 10, 50]
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
for eta in etas:
    v_list.append(maze_asynch.AdaVI_asynch(eta, maze_asynch.Bpi, graph=True))
v_list.append(maze_asynch.VI_asynch(maze_asynch.Bpi, graph=True))
plt.ylabel(r'$||v_k- B_\pi v_k||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.legend()

plt.subplot(1, 2, 2)
for i, eta in enumerate(etas):
    plt.semilogy(range(len(v_list[i])), v_list[i], label=r'ADA-VI asynch, $\eta=$'+str(eta))
plt.semilogy(range(len(v_list[-1])), v_list[-1], label=r'VI asynch')
plt.legend()
plt.ylabel(r'$||v_k- v_\pi||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.suptitle(r'Comparaison de convergence de Ada-VI$^{(V)}_\pi$ et VI$^{(V)}_\pi$ asynchrone')
plt.tight_layout()
plt.subplots_adjust(wspace=0.4)
plt.show()

In [ ]:
v_list = []
etas = [3, 5, 7, 10, 50]
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
for eta in etas:
    v_list.append(maze_asynch.AdaVI_asynch(eta, maze_asynch.Bstar, graph=True))
v_list.append(maze_asynch.VI_asynch(maze_asynch.Bstar, graph=True))
plt.ylabel(r'$||v_k- B_\pi v_k||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.legend()

plt.subplot(1, 2, 2)
for i, eta in enumerate(etas):
    plt.semilogy(range(len(v_list[i])), v_list[i], label=r'ADA-VI asynch, $\eta=$'+str(eta))
plt.semilogy(range(len(v_list[-1])), v_list[-1], label=r'VI asynch')
plt.legend()
plt.ylabel(r'$||v_k- v_*||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.suptitle(r'Comparaison de convergence de Ada-VI$^{(V)}_*$ et VI$^{(V)}_*$ asynchrone')
plt.tight_layout()
plt.subplots_adjust(wspace=0.4)
plt.show()

## Composante par composante


In [ ]:
class Maze_Component(Maze):
    def __init__(self, width=30, height=30, gamma=.9, subsets=5):
        super().__init__(width, height, gamma)

    def AdaVI_component(self, eta, B,  graph=False, espilon=1e-6, max_iter=1e3):
        """
        Adaptive VI algorithm for component per component of the state-value function.
        Be careful, one iteration is one update of all components of the state-value function.

        Parameters
        ----------
        eta : float
            Learning rate
        B : function
            Bellman operator
        graph : bool, optional
            If True, plot the convergence of the algorithm
        espilon : float, optional
            Convergence threshold
        max_iter : int, optional
            Maximum number of iterations

        Returns
        -------
        norm_list : list
            List of the norm of the difference between the state-value function at each iteration and the optimal state-value function
        """
        v_list = [] #list of v at each iteration
        norm_list = [] #list of the norm of the difference between Bv and v
        pi = self.pi
        v = self.v0
        sum_square = np.zeros_like(v) #matrix sum of the square of the matrix difference between Bv and v
        for _ in range(int(max_iter)):
            v_ = v.copy()
            for s in np.argwhere(self.maze != 2): #on all states because if the wall is not updated, the infinite norm of v-Bv become a constant
                diff = B(v_,pi)-v_
                sum_square += (diff)**2
                v_[tuple(s)] = v[tuple(s)] + eta*(diff[tuple(s)])/np.sqrt(sum_square[tuple(s)])
            if np.amax(np.abs(diff)) < espilon:
                print(f'AdaVI converged for eta={eta}')
                break
            elif graph:
                norm_list.append(np.amax(np.abs(diff)))
                v_list.append(v_)
            v = v_.copy()
        else:
            print(f'AdaVI not converged for eta={eta}')
        
        if graph:
            plt.semilogy(range(len(norm_list)), norm_list, label=r'ADA-VI component, $\eta=$'+str(eta))
            return [np.amax(w-v_list[-1]) for w in v_list]
            
    def VI_component(self, B,  graph=False, espilon=1e-6, max_iter=1e3):
        """
        Value Iteration algorithm for component per component of the state-value function.
        Be careful, one iteration is one update of all components of the state-value function.

        Parameters
        ----------
        B : function
            Bellman operator
        graph : bool, optional
            If True, plot the convergence of the algorithm
        espilon : float, optional
            Convergence threshold
        max_iter : int, optional
            Maximum number of iterations
        
        Returns
        -------
        norm_list : list
            List of the norm of the difference between the state-value function at each iteration and the optimal state-value function
        """
        v_list = [] #list of v at each iteration
        norm_list = [] #list of the norm of the difference between Bv and v
        pi = self.pi
        v = self.v0
        for _ in range(int(max_iter)):
            v_ = v.copy()
            for s in np.argwhere(self.maze != 2): #on all states because if the wall is not updated, the infinite norm of v-Bv become a constant
                v_[tuple(s)] = B(v_,pi)[tuple(s)]
            if np.amax(np.abs(v_-v)) < espilon:
                print(f'VI converged')
                break
            elif graph:
                norm_list.append(np.amax(np.abs(v_-v)))
                v_list.append(v_)
            v = v_
        else:
            print(f'VI not converged')
        
        if graph:
            plt.semilogy(range(len(norm_list)), norm_list, label=r'VI component')
            return [np.amax(w-v_list[-1]) for w in v_list]

In [ ]:
np.random.seed(420)
maze_component = Maze_Component(10, 10)

In [ ]:
v_list = []
etas = [3, 5, 7, 10, 50]
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
for eta in etas:
    v_list.append(maze_component.AdaVI_component(eta, maze_component.Bpi, graph=True))
v_list.append(maze_component.VI_component(maze_component.Bpi, graph=True))
plt.ylabel(r'$||v_k- B_\pi v_k||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.legend()

plt.subplot(1, 2, 2)
for i, eta in enumerate(etas):
    plt.semilogy(range(len(v_list[i])), v_list[i], label=r'ADA-VI component, $\eta=$'+str(eta))
plt.semilogy(range(len(v_list[-1])), v_list[-1], label=r'VI component')
plt.legend()
plt.ylabel(r'$||v_k- v_\pi||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.suptitle(r'Comparaison de convergence de Ada-VI$^{(V)}_\pi$ et VI$^{(V)}_\pi$ composante par composante')
plt.tight_layout()
plt.subplots_adjust(wspace=0.4)
plt.show()

In [ ]:
v_list = []
etas = [3, 5, 7, 10, 50]
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
for eta in etas:
    v_list.append(maze_component.AdaVI_component(eta, maze_component.Bstar, graph=True))
v_list.append(maze_component.VI_component(maze_component.Bstar, graph=True))
plt.ylabel(r'$||v_k- B_* v_k||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.legend()

plt.subplot(1, 2, 2)
for i, eta in enumerate(etas):
    plt.semilogy(range(len(v_list[i])), v_list[i], label=r'ADA-VI component, $\eta=$'+str(eta))
plt.semilogy(range(len(v_list[-1])), v_list[-1], label=r'VI component')
plt.legend()
plt.ylabel(r'$||v_k- v_*||_\infty$ (log)')
plt.xlabel('Nombre d\'itérations')
plt.suptitle(r'Comparaison de convergence de Ada-VI$^{(V)}_*$ et VI$^{(V)}_*$ composante par composante')
plt.tight_layout()
plt.subplots_adjust(wspace=0.4)
plt.show()